<a href="https://colab.research.google.com/github/Yaswanthyarra/MicroexpressionClassificationUsingCNNandGeneticAlgorithm/blob/main/geneticAlgoTuningCNNhyperparameters.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd '/content/drive/My Drive/CASMEII'

/content/drive/My Drive/CASMEII


In [4]:
dataset_dir = '/content/drive/My Drive/CASMEII/Cropped-updated/Cropped'

In [5]:
import numpy as np
import pandas as pd
import cv2
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split

# define constants
IMAGE_SIZE = (64, 64)
BATCH_SIZE = 32
NUM_EPOCHS = 60

# load the onset and apex frame indices from the Excel sheet
coding_file = pd.read_excel('/content/drive/My Drive/CASMEII/CASME2-coding-20140508.xlsx', sheet_name='Sheet1')

class_labels = coding_file['Estimated Emotion'].values

# define a function to extract the onset and apex frames for each expression
def extract_frames(sequence_path,sequence, subject_num):
    expression_images = []

    #To obtain the onset and apex frame numbers for the given subject and the corresponding sequence from the xlsheet
    subject_data = coding_file.loc[coding_file['Subject'] == subject_num]
    sequence_data = subject_data.loc[subject_data['Filename'] == sequence]
    onset_frame = sequence_data['OnsetFrame'].iloc[0]
    apex_frame = sequence_data['ApexFrame'].iloc[0]
                                                                                                                                                                 
    #To obtain the path to frames corresponding to the apex and onset frame numbers from the sequence of frames for the subject
    onset_frame_path = os.path.join(sequence_path,'reg_img' + str(onset_frame) + '.jpg')
    apex_frame_path = os.path.join(sequence_path, 'reg_img' + str(apex_frame) + '.jpg')
    print(subject_num,apex_frame_path)
    print(subject_num,onset_frame_path)
   
    #To obtain the apex and apex frames from path
    onset_frame = cv2.imread(onset_frame_path)
    apex_frame = cv2.imread(apex_frame_path)
    onset_frame = cv2.cvtColor(onset_frame, cv2.COLOR_BGR2GRAY)
    apex_frame = cv2.cvtColor(apex_frame, cv2.COLOR_BGR2GRAY)
    
    #To apply histogram equalisation to the frames to to enhance the contrast of an image by redistributing the pixel intensities. 
    onset_frame = cv2.equalizeHist(onset_frame)
    apex_frame = cv2.equalizeHist(apex_frame)

    #Resizing extracted frames
    onset_frame = cv2.resize(onset_frame, IMAGE_SIZE)
    apex_frame = cv2.resize(apex_frame, IMAGE_SIZE)

    #Stacking the apex and onset frames together and and appending to the expressin images list
    expression_images.append(np.stack([onset_frame, apex_frame], axis=-1))
    
    return np.array(expression_images)

# create a data generator for the image data
data_generator = ImageDataGenerator(rescale=1./255)

#Setting up several image augmentation parameters that can be used to generate augmented images 
exp_frame_augmentation = {
    'rotation_range': 10,
    'width_shift_range': 0.1,
    'height_shift_range': 0.1,
    'shear_range': 0.1,
    'zoom_range': 0.1,
    'horizontal_flip': True,
    'brightness_range': [0.5, 1.5],
}


# load the expression folders and extract the onset and apex frames for each subject

#converting the categorical class labels of a classification problem into numerical form
label_dict = {'happiness': 0, 'disgust': 1, 'repression': 2, 'surprise': 3, 'fear': 4, 'others': 6,'sadness':5}

expression_images = []
expression_labels = []

#Looping through all the subjects in the dataset
for subject in os.listdir(dataset_dir):

    subject_num = int(subject[3:])
    subject_dir = os.path.join(dataset_dir, subject)

    #Looping through each sequence for the subject
    for sequence in os.listdir(subject_dir):
        
        sequence_path = os.path.join(subject_dir, sequence)

        #Extracting the onset and apex frames by invoking the extract_frames method
        expression_frames = extract_frames(sequence_path, sequence, subject_num)

        #Obtaining the expression label for the sequence from the xlsheet and appending it to expression_label 
        subject_data = coding_file.loc[coding_file['Subject'] == subject_num]
        sequence_data = subject_data.loc[subject_data['Filename'] == sequence]
        exp_label_category = sequence_data['Estimated Emotion']
        expression_label = label_dict[exp_label_category.values[0]]

        #Adding the extracted frames and the correspoing class label to expression_images and expression_labels
        expression_images.append(expression_frames)
        expression_labels.append(expression_label)

        # Generating augmented images for the fear class
        if expression_label == 4:  # if fear class
            
            for i in range(30):
              print(i,subject_num)

              #Generating the augmented apex and onset frames
              apex_augmented=data_generator.apply_transform(expression_frames[:, :, :, 0], exp_frame_augmentation)
              onset_augmented=data_generator.apply_transform(expression_frames[:, :, :, 1], exp_frame_augmentation)
              
              #Appending the augmented images to expression_images and expression_labels lists
              expression_images.append(np.stack([apex_augmented, onset_augmented], axis=-1))
              expression_labels.append(expression_label)
        
        # Generating augmented images for the sadness class
        if expression_label == 5:  # if sadness class
           
            for i in range(9):
              print(i,subject_num)
              
              #Generating the augmented apex and onset frames
              apex_augmented=data_generator.apply_transform(expression_frames[:, :, :, 0], exp_frame_augmentation)
              onset_augmented=data_generator.apply_transform(expression_frames[:, :, :, 1], exp_frame_augmentation)
              

              #Appending the augmented images to expression_images and expression_labels lists
              expression_images.append(np.stack([apex_augmented, onset_augmented], axis=-1))
              expression_labels.append(expression_label)  

        # generate augmented images for the happiness class
        if expression_label == 0:  # if happiness class
        
            for i in range(1):
              print(i,subject_num)

              #Generating the augmented apex and onset frames
              apex_augmented=data_generator.apply_transform(expression_frames[:, :, :, 0], exp_frame_augmentation)
              onset_augmented=data_generator.apply_transform(expression_frames[:, :, :, 1], exp_frame_augmentation)
              
              #Appending the augmented images to expression_images and expression_labels lists
              expression_images.append(np.stack([apex_augmented, onset_augmented], axis=-1))
              expression_labels.append(expression_label)  

        # Generate augmented images for the repression class
        if expression_label == 2:  # if repression class
          
            for i in range(1):
              print(i,subject_num)
              apex_augmented=data_generator.apply_transform(expression_frames[:, :, :, 0], exp_frame_augmentation)
              onset_augmented=data_generator.apply_transform(expression_frames[:, :, :, 1], exp_frame_augmentation)

              #Appending the augmented images to expression_images and expression_labels lists
              expression_images.append(np.stack([apex_augmented, onset_augmented], axis=-1))
              expression_labels.append(expression_label)  
        
        # Generate augmented images for the surprise class
        if expression_label == 3:  # if surprise class
            
            
            for i in range(1):
              print(i,subject_num)

              #Generating the augmented apex and onset frames
              apex_augmented=data_generator.apply_transform(expression_frames[:, :, :, 0], exp_frame_augmentation)
              onset_augmented=data_generator.apply_transform(expression_frames[:, :, :, 1], exp_frame_augmentation)

              #Appending the augmented images to expression_images and expression_labels lists
              expression_images.append(np.stack([apex_augmented, onset_augmented], axis=-1))
              expression_labels.append(expression_label)  


        

        
# convert the data to numpy arrays 
expression_images = np.concatenate(expression_images, axis=0)
expression_labels = np.array(expression_labels)

#To obtain the number of class labels
num_classes = len(np.unique(expression_labels))

#Stratified Spliting of data into training and testing sets
train_images, test_images, train_labels, test_labels = train_test_split(expression_images, expression_labels, test_size=0.2,stratify=expression_labels,  random_state=42)



25 /content/drive/My Drive/CASMEII/Cropped-updated/Cropped/sub25/EP09_02/reg_img101.jpg
25 /content/drive/My Drive/CASMEII/Cropped-updated/Cropped/sub25/EP09_02/reg_img76.jpg
25 /content/drive/My Drive/CASMEII/Cropped-updated/Cropped/sub25/EP10_01/reg_img129.jpg
25 /content/drive/My Drive/CASMEII/Cropped-updated/Cropped/sub25/EP10_01/reg_img101.jpg
25 /content/drive/My Drive/CASMEII/Cropped-updated/Cropped/sub25/EP03_01/reg_img81.jpg
25 /content/drive/My Drive/CASMEII/Cropped-updated/Cropped/sub25/EP03_01/reg_img56.jpg
25 /content/drive/My Drive/CASMEII/Cropped-updated/Cropped/sub25/EP03_02/reg_img101.jpg
25 /content/drive/My Drive/CASMEII/Cropped-updated/Cropped/sub25/EP03_02/reg_img51.jpg
25 /content/drive/My Drive/CASMEII/Cropped-updated/Cropped/sub25/EP10_10/reg_img91.jpg
25 /content/drive/My Drive/CASMEII/Cropped-updated/Cropped/sub25/EP10_10/reg_img61.jpg
0 25
25 /content/drive/My Drive/CASMEII/Cropped-updated/Cropped/sub25/EP18_04f/reg_img116.jpg
25 /content/drive/My Drive/CASME

In [6]:

!pip install deap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 6.4 MB/s eta 0:00:00


In [12]:
# Import required packages
from keras.models import Sequential
from keras.layers import Conv3D, MaxPooling3D, Flatten, Dense, Dropout
from keras.optimizers import Adam
import numpy as np
import random

#DEAP (Distributed Evolutionary Algorithms in Python) a powerful evolutionary computation framework for Python.
from deap import algorithms, base, creator, tools


# Defining the fitness function
def fitness_function(individual, hall_of_fame):

    # Extract the hyperparameters from the individual
    learning_rate, batch, dropout, Epochs = individual
    print(learning_rate, batch, dropout, Epochs)

    # Create a Sequential model object
    model = Sequential([
        
    # Add a 2D convolutional layer with 32 filters of size (3,3), using the 'relu' activation function,
    # and an input shape of (IMAGE_SIZE[0], IMAGE_SIZE[1], 2) for 2-channel input images
    Conv2D(32, (3,3), activation='relu', input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 2)),
    
    # Add a 2D max pooling layer with a pool size of (2,2)
    MaxPooling2D((2,2)),
    
    # Add another 2D convolutional layer with 64 filters of size (3,3), using the 'relu' activation function
    Conv2D(64, (3,3), activation='relu'),
    
    # Add another 2D max pooling layer with a pool size of (2,2)
    MaxPooling2D((2,2)),
    
    # Add another 2D convolutional layer with 128 filters of size (3,3), using the 'relu' activation function
    Conv2D(128, (3,3), activation='relu'),
    
    # Add another 2D max pooling layer with a pool size of (2,2)
    MaxPooling2D((2,2)),
    
    # Flatten the output of the previous layer to a 1D vector
    Flatten(),
    
    # Add a dense layer with 128 units and 'relu' activation function
    Dense(128, activation='relu'),
    
    # Add a dropout layer with a dropout rate of `dropout`
    Dropout(abs(dropout)),
    
    # Add a dense layer with `num_classes` units and 'softmax' activation function
    Dense(num_classes, activation='softmax')
])

    
    
    # Compile the model with the given hyperparameters
    optimizer = Adam(lr=abs(learning_rate))
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    #Training the model
    model.fit(train_images, train_labels, epochs=abs(int(Epochs)), batch_size=abs(int(batch)), validation_data=(test_images, test_labels))

    #Evaluating the model performance by determining the loss_value and accuracy
    loss_value, accuracy = model.evaluate(test_images,test_labels, verbose=0)
    print(individual,accuracy)

    # Use the selBest() function from the tools module of the DEAP framework to select the best individuals
    hall_of_fame = tools.selBest(hall_of_fame.items, k=HALL_OF_FAME_SIZE)
    print(f'Hall of fame: {hall_of_fame}')

    # Return the accuracy as the fitness value
    return accuracy,

# Define the genetic algorithm parameters
POPULATION_SIZE = 5
P_CROSSOVER = 0.7
P_MUTATION = 0.3
MAX_GENERATIONS = 5
HALL_OF_FAME_SIZE = 3
OFFSPRING_SIZE=5

# Define the initial population
individuals =  [[0.001, 32, 0.2, 24], 
               [0.0003, 32, 0.2, 30],
               [0.0005, 64, 0.3, 50], 
               [0.002, 16, 0.4, 45], 
               [0.0008, 32, 0.3, 40]
              ]




# Create a new fitness class 'FitnessMax' based on the base class 'Fitness' from the DEAP library,
# with a weight of 1.0. This means that we want to maximize the fitness value.
creator.create("FitnessMax", base.Fitness, weights=(1.0,))

# Create a new class 'Individual' that inherits from the built-in list class, with a fitness attribute
# that is an instance of the 'FitnessMax' class created earlier.
creator.create("Individual", list, fitness=creator.FitnessMax)

# Create a population of individuals by creating a new 'Individual' object for each set of
# parameters in the 'individuals' list.
population = [creator.Individual(ind) for ind in individuals]

# Create a hall of fame object, which is a container that keeps track of the best individuals
# found during the evolution process.
hall_of_fame = tools.HallOfFame(HALL_OF_FAME_SIZE)

# Create a statistics object that will be used to keep track of the maximum, average, and standard
# deviation of the fitness values in the population.
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register('max', np.max)
stats.register('avg', np.mean)
stats.register('std', np.std)

# Create a toolbox object, which is a container that groups together all the functions needed
# for the evolution process.
toolbox = base.Toolbox()


# Register the fitness function to be used to evaluate the fitness of individuals in the population.
# The hall of fame object is also passed as an argument, so that the best individuals found so far
# can be tracked.
toolbox.register('evaluate', lambda ind: fitness_function(ind, hall_of_fame))

# Register the selection operator to be used in the evolution process. In this case, we use the
# tournament selection method with a tournament size of 3.
toolbox.register('select', tools.selTournament, tournsize=3)

# Register the mating operator to be used in the evolution process. In this case, we use the
# blend crossover method with a blending parameter of 0.5.
toolbox.register('mate', tools.cxBlend, alpha=0.5)

# Register the mutation operators to be used in the evolution process. We register two mutation
# operators: one for floating-point numbers, and one for integers. The Gaussian mutation operator
# is used for floating-point numbers, with a mean of 0 and a standard deviation of 0.1. The uniform
# integer mutation operator is used for integers, with a range of 1 to 10 and a probability of 0.1.
toolbox.register('mutate_float', tools.mutGaussian, mu=0, sigma=0.1, indpb=0.1)
toolbox.register('mutate_int', tools.mutUniformInt, low=1, up=10, indpb=0.1)

#Implementing a function to determine which mutation operator to use
def mutate_individual(individual):
    for i in range(len(individual)):
        if isinstance(individual[i], int):
            individual[i] = int(toolbox.mutate_int([individual[i]])[0][0])
        else:
            individual[i] = toolbox.mutate_float([individual[i]])[0][0]
    return individual,

toolbox.register('mutate', mutate_individual)

# Use the DEAP evolutionary algorithm eaMuPlusLambda to perform a mu + lambda evolution strategy
# - The population argument is the initial population of candidate solutions to evolve
# - The toolbox argument is a DEAP toolbox that contains the genetic operators and other configuration settings
# - mu is the number of individuals to select for the next generation (the "mu" in mu + lambda)
# - lambda_ is the number of offspring to generate for the next generation (the "lambda" in mu + lambda)
# - cxpb is the probability of performing crossover between two individuals
# - mutpb is the probability of performing mutation on an individual
# - ngen is the maximum number of generations to run the algorithm for
# - stats is a DEAP statistics object that collects data about the evolution process
# - halloffame is a DEAP hall of fame object that keeps track of the best individuals over the generations
# - verbose is a boolean flag that controls the verbosity of the output (True = print progress messages)
# The function returns the final population and a logbook containing the statistics of the evolution process.

population, logbook = algorithms.eaMuPlusLambda(population, toolbox, mu=POPULATION_SIZE, lambda_=OFFSPRING_SIZE, cxpb=P_CROSSOVER, mutpb=P_MUTATION, ngen=MAX_GENERATIONS, stats=stats, halloffame=hall_of_fame, verbose=True)

#Obtaining the best individual
best_individual = hall_of_fame[0]
print(f'Best individual: {best_individual}, fitness: {best_individual.fitness.values[0]}')



0.001 32 0.2 24
Epoch 1/24
12/12 [==============================] - 3s 223ms/step - loss: 22.0167 - accuracy: 0.1545 - val_loss: 1.8351 - val_accuracy: 0.3441
Epoch 2/24
12/12 [==============================] - 4s 312ms/step - loss: 1.9786 - accuracy: 0.2575 - val_loss: 1.6264 - val_accuracy: 0.3656
Epoch 3/24
12/12 [==============================] - 3s 263ms/step - loss: 1.6155 - accuracy: 0.4038 - val_loss: 1.3906 - val_accuracy: 0.4516
Epoch 4/24
12/12 [==============================] - 2s 202ms/step - loss: 1.2878 - accuracy: 0.5014 - val_loss: 1.1871 - val_accuracy: 0.5269
Epoch 5/24
12/12 [==============================] - 2s 200ms/step - loss: 1.0284 - accuracy: 0.6070 - val_loss: 0.9457 - val_accuracy: 0.6022
Epoch 6/24
12/12 [==============================] - 2s 202ms/step - loss: 0.8473 - accuracy: 0.6775 - val_loss: 0.7727 - val_accuracy: 0.7312
Epoch 7/24
12/12 [==============================] - 3s 275ms/step - loss: 0.6612 - accuracy: 0.7751 - val_loss: 0.7326 - val_accura

In [14]:
 for i in hall_of_fame:
   print(i,i.fitness.values[0])

[0.0006651508397586441, 17.841842195711813, 0.29999999999999993, 52.73272478423032] 0.9032257795333862
[0.0008, 32, 0.3, 40] 0.8924731016159058
[0.001, 32, 0.2, 24] 0.8817204236984253


In [16]:
#Displaying generation wise statistics
print()
'''
for gen, stats in enumerate(logbook):
    print(f"Generation {gen}:")
    print(f"  Maximum fitness: {stats['max']:.2f}")
    print(f"  Average fitness: {stats['avg']:.2f}")
    print(f"  Standard deviation: {stats['std']:.2f}")'''

'\nfor gen, stats in enumerate(logbook):\n    print(f"Generation {gen}:")\n    print(f"  Maximum fitness: {stats[\'max\']:.2f}")\n    print(f"  Average fitness: {stats[\'avg\']:.2f}")\n    print(f"  Standard deviation: {stats[\'std\']:.2f}")'